# Cálculo explícito de los polinomios de Hermite utilizando la fórmula de Rodrigues
**Juan Antonio Villegas Recio**
> Nota: Este Notebook debe ser ejecutado utilizando un 'kernel' de SageMath, preferiblemente con una versión igual o superior a la 9.3.

El objetivo es implimentar un método de cálculo explícito de los polinomios de Hermite utilizando la fórmula de Rodrigues. También se harán algunas mediciones de tiempos de ejecución para compararlas con los tiempos calculados con la RRTT.

Los polinomios de Hermite son ortogonales respecto a la función peso $\rho(x) = e^{-x^2}$ y verifican la ecuación de Pearson con $\sigma(x)=1$, $x\in\mathbb R$. Por tanto, la fórmula de Rodrigues de los polinomios de Hermite, tomando por convenio $B_n=(-1)^n$ es
$$
H_n(x)=(-1)^n e^{x^2}\frac{d^n}{dx^n}e^{x^{-2}}
$$
Creamos una función que recree esta fórmula, devolviendo un polinomio

In [1]:
def Rodrigues_H(n):
    Bn = (-1)**n
    rho_inv = e**(x**2)
    der = derivative(e**(-x**2),x,n)
    return (Bn*rho_inv*der).full_simplify()

Veámos los primeros 11 polinomios de Hermite, esta vez estandarizados como suelen ser presentados:

In [2]:
Npol = 11
for i in range(Npol):
    print(str(i) + ": " + str(Rodrigues_H(i)))

0: 1
1: 2*x
2: 4*x^2 - 2
3: 8*x^3 - 12*x
4: 16*x^4 - 48*x^2 + 12
5: 32*x^5 - 160*x^3 + 120*x
6: 64*x^6 - 480*x^4 + 720*x^2 - 120
7: 128*x^7 - 1344*x^5 + 3360*x^3 - 1680*x
8: 256*x^8 - 3584*x^6 + 13440*x^4 - 13440*x^2 + 1680
9: 512*x^9 - 9216*x^7 + 48384*x^5 - 80640*x^3 + 30240*x
10: 1024*x^10 - 23040*x^8 + 161280*x^6 - 403200*x^4 + 302400*x^2 - 30240


Recordemos los tiempos de cálculo que nos ofrecía el método que utilizaba la RRTT, los cuales eran elevados y crecían muy rápido con $n$. Comprobaremos si el cálculo mediante la fórmula de Rodrigues mejora o no a la RRTT. Para ello, y de manera análoga, haremos $10$ ejecuciones independientes del cálculo de cada polinomio y, mediante la media de los $10$ tiempos de ejecución encontraremos una estimación bastante acertada del tiempo medio de ejecución del cálculo de cada polinomio.

In [3]:
import time
Nejecuciones = 10
tiempos_calculo = [[] for _ in range(Npol)]
for i in range(Nejecuciones):
    for j in range(Npol):
        start_time = time.time()
        Rodrigues_H(j)
        tiempos_calculo[j].append(time.time() - start_time)

for i in range(Npol):
    print(str(i) + ": " + str(sum(tiempos_calculo[i])/Nejecuciones))

0: 0.0054595947265625
1: 0.005772542953491211
2: 0.010381221771240234
3: 0.010808849334716797
4: 0.012038230895996094
5: 0.013904762268066407
6: 0.015523004531860351
7: 0.015167832374572754
8: 0.01721677780151367
9: 0.017970967292785644
10: 0.020062971115112304


Como vemos, estos tiempos de cálculo son muchísimo menores que los calculados para la RRTT. De hecho, obsérvese el tiempo que tarda el programa en calcular el polinomio de Hermite de grado 500:

In [4]:
start_time = time.time()
Rodrigues_H(500)
time.time() - start_time

13.950093984603882

Teniendo en cuenta que la RRTT tardaba aproximadamente un minuto en calcular el polinomio de grado 15, la mejora es sustancial.

La siguiente celda calcula los polinomios de Hermite utilizando la fórmula Rodrigues y utilizando la funcionalidad nativa de Sage, de tal forma que podamos comparar ambos métodos. Se ejecuta 10 veces cada grado con cada método y se calcula el tiempo de ejecución promedio de las 10 ejecuciones.

In [5]:
import time
Nejecuciones = 10
tiempos_calculo_Rodrigues = [[] for _ in range(Npol)]
tiempos_calculo_nativa = [[] for _ in range(Npol)]

for i in range(Nejecuciones):
    for j in range(Npol):
        start_time = time.time()
        Rodrigues_H(j)
        tiempos_calculo_Rodrigues[j].append(time.time() - start_time)
        
        start_time = time.time()
        hermite(j,x)
        tiempos_calculo_nativa[j].append(time.time() - start_time)

for i in range(Npol):
    print(str(i) + "| " + str(sum(tiempos_calculo_Rodrigues[i])/Nejecuciones) + \
          " | " + str(sum(tiempos_calculo_nativa[i])/Nejecuciones))

0| 0.005958724021911621 | 4.365444183349609e-05
1| 0.007328152656555176 | 5.514621734619141e-05
2| 0.010941219329833985 | 5.8817863464355466e-05
3| 0.011658930778503418 | 5.893707275390625e-05
4| 0.013392281532287598 | 8.678436279296875e-05
5| 0.014207077026367188 | 5.550384521484375e-05
6| 0.01648435592651367 | 6.401538848876953e-05
7| 0.016787505149841307 | 6.310939788818359e-05
8| 0.019718575477600097 | 6.42538070678711e-05
9| 0.01858944892883301 | 6.23464584350586e-05
10| 0.021929621696472168 | 6.678104400634766e-05


Vemos que, aunque la fórmula de Rodrigues rinde mucho mejor que la RRTT, la funcionalidad nativa de Sage es aún más rápida.